# All-in Algorithm 

This notebook demonstrates the performance of the all-in algorithm proposed in the paper, where relevant results can be found in Table 2, Section 5.1.2.

## 1. Preloading Packages and Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import cvxpy as cp
from scipy.optimize import linprog
from scripts.estimation import *
from scripts.utils import *

## 2. Synthetic Data Generation
We generate the synthetic data from a Poisson process with an underlying MNL model as rider's choice probability. In this example, all arrival locations are generated from a uniform distribution within a square field inside the location bound.

In [2]:
rand_seed = 1
num_position = 5
bike_num = 20
lambd = 10
grid_size = 10
beta0 = 1
beta1_true = -1
T = 100
loc_bound = 5
bike_num,num_records,book_bike,book_index,dist, \
bike_loc,all_period,num_booked,cand_loc,true_loc, \
position_weight=gen_sync(rand_seed,num_position,bike_num,lambd,grid_size,beta0=beta0,beta1_true=beta1_true,T=T,loc_bound=loc_bound)

## 3. Implementation of the All-in algorithm

The following snippets implement the all-in algorithm under the synthetic data generated above. In short, the all-in algorithm enumerates a large set of all possible candidate locations to estimate the rider location set $\mathcal{L}$. Here, we assume that the values of the MNL model parameters $\beta_0$ and $\beta_1$ are given.

In [3]:
# All-in algorithm (beta1 given)

np.random.seed(seed=1)
thres = 0.01
select_index = np.arange(grid_size**2)
w = findw_EM(select_index.shape[0],cand_loc[0,select_index],beta0,bike_num,num_records,bike_loc,book_bike,num_booked,book_index,all_period,T)

In [4]:
# Location trimming
select_index = np.where(w>thres)[0]
w_select = w[select_index]
w_select = findw_EM(select_index.shape[0],cand_loc[0,select_index],beta0,bike_num,num_records,bike_loc,book_bike,num_booked,
             book_index,all_period,T)
wasser_dist_disc = find_wasserstein(cand_loc[0,select_index],true_loc,w_select,position_weight)
trimmed_loc_num = cand_loc[0,select_index].shape[0]
trimmed_lkd = findlkd_no_constraint(trimmed_loc_num,caldist(cand_loc[0,select_index],bike_loc,bike_num),beta0,
                      np.repeat(-1,trimmed_loc_num).reshape(-1,1),w_select,bike_num,num_records,book_bike,book_index,num_booked,all_period)
print('The Wasserstein distance is:',wasser_dist_disc[0])
print('The log-likelihood value is:',trimmed_lkd)
print('The BIC value is:', -trimmed_lkd+0.5*trimmed_loc_num*np.log(num_booked))

The Wasserstein distance is: 2.403561532050001
The log-likelihood value is: -1777.8913740351745
The BIC value is: 1851.0971271988524


## 4. Jointly estimating $\boldsymbol{w}$ and $\boldsymbol{\beta}_1$
We extend the all-in algorithm to jointly estimate the weight vector $\boldsymbol{w}$ and the slope $\boldsymbol{\beta}_1$ in the MNL model when the slope $\boldsymbol{\beta}_1$ is not given. Here, we assume that $\boldsymbol{\beta}_0$ is known and 
$\boldsymbol{\beta}_1$ is to be estimated. We further assume that $\boldsymbol{\beta}_0$ and $\boldsymbol{\beta}_1$ have the same value across all locations. All other setups are the same as before.

In [5]:
# All-in algorithm (beta1 unknown)
np.random.seed(seed=1)
beta1_cur = np.random.uniform(-10,0)
loc_old = cand_loc[-1]
dist_old = caldist(loc_old,bike_loc,bike_num)
beta1_cur, w = findbetaw_EM(grid_size**2,cand_loc,beta0,
                            beta1_cur,bike_num,num_records,bike_loc,book_bike,num_booked,book_index,all_period,T)

In [6]:
# Location trimming
select_index = np.where(w>thres)[0]
w_select = w[select_index]
beta1_em, w_select = findbetaw_EM(select_index.shape[0],cand_loc[:,select_index],beta0,
                                  beta1_cur,bike_num,num_records,bike_loc,book_bike,num_booked,book_index,all_period,T)
wasser_dist_disc = find_wasserstein(cand_loc[0,select_index],true_loc,w_select,position_weight)
print('The Wasserstein distance is:',wasser_dist_disc[0])
print('The MAPE for beta1 is:',np.abs(beta1_em-beta1_true)/-beta1_true)

The Wasserstein distance is: 2.667006964429996
The MAPE for beta1 is: 0.29964829426228534
